In [23]:
# import all of the libraries
import pandas as pd
import numpy as np
import folium
from folium import plugins
import ipywidgets
import geocoder
from branca.element import Figure
import plotly.graph_objs as go
import plotly.offline as py
import plotly.express as px
from plotly.subplots import make_subplots
import dash
import dash_core_components as dcc
import dash_html_components as html
#import chart_studio.plotly as py
#import matplotlib.pyplot as plt
#import seaborn as sns
#%matplotlib inline

In [5]:
df = pd.read_csv('data/1976-2016-president.csv')
df.head()

,year,state,state_po,state_fips,state_cen,state_ic,office,candidate,party,writein,candidatevotes,totalvotes,version,notes
0,1976,Alabama,AL,1,63,41,US President,"Carter, Jimmy",democrat,False,659170,1182850,20171015,NaN
1,1976,Alabama,AL,1,63,41,US President,"Ford, Gerald",republican,False,504070,1182850,20171015,NaN
2,1976,Alabama,AL,1,63,41,US President,"Maddox, Lester",american independent party,False,9198,1182850,20171015,NaN
3,1976,Alabama,AL,1,63,41,US President,"Bubar, Benjamin """"Ben""""",prohibition,False,6669,1182850,20171015,NaN
4,1976,Alabama,AL,1,63,41,US President,"Hall, Gus",communist party use,False,1954,1182850,20171015,NaN


In [53]:
df['pct_vote'] = (df['candidatevotes'] / df['totalvotes']).astype('float64')

#democratic-farmer-labor is a special case for MN
df.replace({'party': 'democratic-farmer-labor'}, 'democrat', inplace = True)

# replace all minor parties with 'other'
df.loc[~df['party'].isin(['republican','democrat']),'party'] = 'other'

# retrieve corresponding id for candidate with max votes by year and state
winner = df.groupby(['year','state_po'], as_index = False).agg({'candidatevotes':'idxmax',\
                                                              'pct_vote': lambda g: sum(df.loc[g.index].query("party == 'democrat'").pct_vote)})

# add column for corresponding party for max votes for candidate
winner['party'] = [df.loc[id,'party'] for id in winner['candidatevotes']]

# map parties to colors for plotting later on. 
winner['party_id'] = winner['party'].map({'democrat': 1.0, 'republican': 0.0}) 
winner.head()

,year,candidate,candidatevotes
0,1976,"Anderson, Thomas J.",21325
1,1976,"Bubar, Benjamin """"Ben""""",6669
2,1976,"Camejo, Peter",17802
3,1976,"Carter, Jimmy",3742284
4,1976,"Ford, Gerald",3882244


In [57]:
#national winner
national_winner = df.groupby(['year']).agg({'candidatevotes':'idxmax'})
national_winner['candidate'] = [df.loc[id,'candidate'] for id in national_winner['candidatevotes']]
national_winner.head()

,candidatevotes,candidate
year,,
1976,23,"Ford, Gerald"
1980,362,"Reagan, Ronald"
1984,721,"Reagan, Ronald"
1988,1041,"Bush, George H.W."
1992,1332,"Clinton, Bill"


In [ ]:
# https://python-visualization.github.io/folium/quickstart.html

url = 'https://raw.githubusercontent.com/python-visualization/folium/master/examples/data'
state_geo = f'{url}/us-states.json'

m = folium.Map(location=[48, -102], zoom_start=3)

folium.Choropleth(
    geo_data=state_geo,
    name='choropleth',
    data = winner[winner['year'] == 2016],
    columns =['state_po', 'pct_vote'],
    key_on ='feature.id',
    fill_color = 'RdBu',
    fill_opacity = 0.7,
    line_opacity = 0.2,
    #bins = 3,
    legend_name='Party'
).add_to(m)

folium.LayerControl().add_to(m)

m

In [12]:
#https://support.sisense.com/hc/en-us/community/posts/360038301533-Plotly-Choropleth-With-Slider-Map-Charts-Over-Time-
min_year = 1976

# your color-scale
scl = [[0.0, 'red'],[.50, '#5700c9'], [1,'#0515a8']]
#scl = [[0.0, 'red'], [1,'blue']]
data_slider = []
for year in winner['year'].unique():
    df_segmented =  winner[(winner['year']== year)]
    data_each_yr = dict(
                        type='choropleth',
                        locations = df_segmented['state_po'],
                        z=df_segmented['pct_vote'].astype(float),
                        locationmode='USA-states',
                        colorscale = scl,
                        colorbar= {'title':'Party'})

    data_slider.append(data_each_yr)

steps = []

for i in range(len(data_slider)):
    step = dict(method='restyle',
                args=['visible', [False] * len(data_slider)],
                label='Year {}'.format(i*4 + min_year))
    step['args'][1][i] = True
    steps.append(step)

sliders = [dict(active=0, pad={"t": 1}, steps=steps)]

layout = dict(title ='Presidential Wins', geo=dict(scope='usa',
                       projection={'type': 'albers usa'}),
              sliders=sliders)

fig = dict(data=data_slider, layout=layout)
py.plot(fig)

'temp-plot.html'

In [61]:
min_year = 1976
fig = make_subplots(rows=2, cols=2, specs=[[{"type": "pie"},{"type": "choropleth","rowspan":2}], 
                                           [{"type": "indicator"},None ]],
                   subplot_titles=("National Split", "% Democratic Vote"), )

scl = [[0.0, 'red'],[.50, '#5700c9'], [1,'#0515a8']]


data_slider = []
for year in winner['year'].unique():
    
    # Choropleth for each year
    df_segmented =  winner[winner['year']== year]
    fig.append_trace(
        go.Choropleth(
            locations = df_segmented['state_po'],
            z=df_segmented['pct_vote'].astype(float),
            locationmode='USA-states',
            colorscale = scl,
            colorbar= {'title':'Party'}, text = df_segmented['party'], 
            hoverinfo = ["location", "z", "text"]),row = 1, col = 2)

    # Pie Chart for each year
    full_election_year = df[df['year'] == year].groupby('party',as_index = False).agg({'candidatevotes':'sum'}).sort_values(by = 'party')
    fig.append_trace(go.Pie(values = full_election_year['candidatevotes'], 
           labels = full_election_year['party'], marker = {'colors':['blue','green','red']}), row = 1, col = 1)
    
    # Card
    fig.append_trace(
        go.Indicator(
            mode = "number",
            value = year,
#                 title = {"text": "Accounts<br><span style='font-size:0.8em;color:gray'>Subtitle</span><br><span style='font-size:0.8em;color:gray'>Subsubtitle</span>"},
#                 delta = {'reference': 400, 'relative': True},
            title = {"text": f"Popular Vote Winner <br><span style='font-size:0.8em;color:gray'>{national_winner.loc[year,'candidate']} </span>"}), row =2, col = 1)
    
steps = []
for i in range(0,len(fig.data),3):
    step = dict(method='restyle',
                args=['visible', [False] * len(fig.data)],
                label='Year {}'.format(int(i*(4/3) + min_year)))
    step["args"][1][i:i+3] = [True, True, True]
    steps.append(step)

sliders = [dict(active=0, pad={"t": 1}, steps=steps)]

fig.update_layout(sliders=sliders, title ='Presidential Wins')
fig.update_geos(scope='usa', projection={'type': 'albers usa'})
fig.update_layout(legend= {'yanchor':"top",  'y':0.99, 'xanchor':"left",'x':0.01})
#fig.update_layout(showlegend = False)
#fig.update_layout(width=1000,height=800)
py.plot(fig)

'temp-plot.html'

In [13]:
scl = [[0.0, 'red'],[.50, '#5700c9'], [1,'#0515a8']]
px.choropleth(winner,locations='state_po', locationmode="USA-states", color='pct_vote', 
              scope="usa", animation_frame ="year", color_continuous_scale = scl)

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on
